# kriging module

Available classes:

- ArealKriging: Class performs kriging of areas with point support data within those polygons,
- CentroidPoissonKriging: Class performs centroid-based Poisson Kriging of areas,
- Kriging: Class for kriging interpolation of the unknown values at a given location (position).

## ```ArealKriging```

### Class initialization

```python
pyinterpolate.kriging.ArealKriging(
    semivariogram_model,
    known_areas,
    known_areas_points,
    kriging_type='ata')
```

Class performs kriging of areas with point support data within those polygons.

INITIALIZATION PARAMS:

- **semivariogram_model**: (_TheoreticalSemivariogram_) Theoretical Semivariogram used for data interpolation,
- **known_areas**: (_numpy array_) areas in the form:

```python
[area_id, polygon, centroid x, centroid y, value]
```
- **known_areas_points**: (_numpy array_) points within areas in the form:

```python
[
    area_id,
    [point_position_x, point_position_y, value]
]
```
- **kriging_type**: (_str_) default ```'ata'```;

```'ata'``` - area to area poisson kriging,
```'atp'``` - area to point poisson kriging.

### Class public methods:

- **predict**: predicts area value in an unknown location based on the area-to-area or area-to-point Poisson Kriging,
- **regularize_data**: regularizes whole dataset and creates new values and error maps based on the kriging type.

---

### ```ArealKriging.predict()```

```python
ArealKriging.predict(self,
                     unknown_area_points,
                     number_of_neighbours,
                     max_search_radius)
```

Function predicts areal value in an unknown location based on the area-to-area or area-to-point Poisson Kriging.

INPUT:

- **unknown_area_points**: (_numpy array_) points within an unknown area in the form:

```python
[
    area_id,
    [point_position_x, point_position_y, value]
]
```
- **number_of_neighbours**: (_int_) minimum number of neighbours to include in the algorithm,
- **max_search_radius**: (_float_) maximum search radius (if number of neighbours within this search radius is smaller than ```number_of_neighbours``` parameter then additional neighbours are included up to ```number_of_neighbors```).

OUTPUT:

- prediction, error, estimated mean, weights:

```python
[
    value_in_unknown_location,
    error,
    estimated_mean,
    weights
]
```

---

### ```ArealKriging.regularize_data()```

```python
ArealKriging.regularize_data(self,
                             number_of_neighbours,
                             max_search_radius,
                             data_crs="EPSG:4326")
```

Function regularizes whole dataset and creates new values and error maps based on the kriging type.

If chosen type is area to area then function returns ```Geopandas GeoDataFrame``` with area id, areal geometry, estimated value, estimated prediction error, RMSE of prediction.

If chosen type is area to point then function returns ```Geopandas GeoDataFrame``` with area id, point coordinates, estimated value, estimated prediction error, RMSE of areal prediction.

Function do not predict unknown values, areas with ```NaN``` are skipped.

INPUT:

- **number_of_neighbours**: (_int_) minimum number of neighbours to include in the algorithm,
- **max_search_radius**: (_float_) maximum search radius (if number of neighbours within this search radius is smaller than number_of_neighbours parameter then additional neighbours are included up to number_of_neighbours),
- **data_crs**: (_str_) data crs, look into: https://geopandas.org/projections.html

OUTPUT:

- regularized dataset (```GeoPandas GeoDataFrame``` object).

***

## ```CentroidPoissonKriging```

### Class initialization

```python
pyinterpolate.kriging.CentroidPoissonKriging(
    semivariogram_model,
    known_areas,
    known_areas_points)
```

Class performs centroid-based Poisson Kriging of areas.

INITIALIZATION PARAMS:

- **semivariogram_model**: (_TheoreticalSemivariogram_) Theoretical Semivariogram used for data interpolation,
- **known_areas**: (_numpy array_) areas in the form:

```python
[area_id, polygon, centroid x, centroid y, value]
```
- **known_areas_points**: (_numpy array_) points within areas in the form:

```python
[
    area_id,
    [point_position_x, point_position_y, value]
]
```

### Class public methods:

- **predict**: predicts areal value in a unknown location based on the centroid-based Poisson Kriging.

---

### ```CentroidPoissonKriging.predict()```

```python
CentroidPoissonKriging.predict(self,
    unknown_area,
    unknown_area_points,
    number_of_neighbours,
    max_search_radius,
    weighted,
    test_anomalies=True)
```

Function predicts areal value in a unknown location based on the centroid-based Poisson Kriging.

INPUT:

- **unknown_area**: (_numpy array_) unknown area data in the form:

```python
[area_id, polygon, centroid x, centroid y]
```
- **unknown_area_points**: (_numpy array_) points within an unknown area in the form:

```python
[
    area_id,
    [point_position_x, point_position_y, value]
]
```
- **number_of_neighbours**: (_int_) minimum number of neighbours to include in the algorithm,
- **max_search_radius**: (_float_) maximum search radius (if number of neighbours within this search radius is smaller than ```number_of_neighbours``` parameter then additional neighbours are included up to the ```number_of_neighbours```).
- **weighted**: (_bool_) distances weighted by population (```True```) or not (```False```),
- **test_anomalies**: (_bool_) check if weights are negative.

OUTPUT:

- prediction, error, estimated mean, weights:

```python
[
    value_in_unknown_location,
    error,
    estimated_mean,
    weights
]
```

***

## ```Krige```

### Class initialization

```python
pyinterpolate.kriging.Krige(semivariogram_model,
                            known_points)
```

Class for kriging interpolation of the unknown values at a given location (position). Class takes two arguments during the initialization: semivariogram model and known points list.

Class methods may raise ```ValueError``` if estimated value is below 0. You may use ```try: ... except: ...``` statement to overwrite those values with some constant or ```NaN``` or you could use different semivariogram model. Sometimes this problem is related to the input data, especially to the clustered groups of points. In this case aggregate those clusters and then estimate semivariogram and perform kriging.

INITIALIZATION PARAMS:

- **semivariogram_model**: (_TheoreticalSemivariogram_) Theoretical Semivariogram used for data interpolation,
- **known_points**: (_numpy array_) dataset with known values and locations. Each column should represent different dimension and the last column represents values example: 

```python
[
    [dim_x1, dim_y1, val_1],
    [dim_x2, dim_y2, val_2]
]
```

### Class public methods:

- **ordinary_kriging**: predicts value at unknown location with Ordinary Kriging technique,
- **simple_kriging**: predicts value at unknown location with Simple Kriging technique.

---

### ```Krige.ordinary_kriging()```

```python
Krige.ordinary_kriging(self,
                       unknown_location,
                       number_of_neighbours,
                       test_anomalies=True)
```

Function predicts value at unknown location with Ordinary Kriging technique.

INPUT:

- **unknown_location**: (_tuple_) position of unknown location,
- **number_of_neighbours**: (_int_) number of the closest locations to the unknown position which should be included in the modeling,
- **test_anomalies**: (_bool_) check if weights are negative.

OUTPUT:

- predicted, error, estimated mean, weights:
            
```python
[
    value_in_unknown_location,
    error,
    estimated_mean,
    weights
]
```

---

### ```Krige.simple_kriging()```

```python
Krige.simple_kriging(self,
                     unknown_location,
                     number_of_neighbours,
                     mu=None,
                     test_anomalies=True)
```

Function predicts value at unknown location with Simple Kriging technique.

INPUT:

- **unknown_location**: (_tuple_) position of unknown location,
- **number_of_neighbours**: (_int_) number of the closest locations to the unknown position which should be included in the modeling,
- **mu**: (_float_) global mean which should be known before processing. If not given then it is calculated from the sample but then it may cause a relative large errors (this mean is expectation of the random field, so without knowledge of the ongoing processes it is unknown),
- **test_anomalies**: (_bool_) check if weights are negative.

OUTPUT:

- predicted, error, estimated mean, weights:
            
```python
[
    value_in_unknown_location,
    error,
    mean,
    weights
]
```